In [3]:
import os
import unicodedata

dir_root = 'C:/capstone/data'
image_root = dir_root + "/train/images"
label_root = dir_root + "/train/labels"
val_image_root = dir_root + '/validation/images'
val_label_root = dir_root + '/validation/labels'

# 클래스 이름과 클래스 인덱스 매핑
product_list = os.listdir(image_root)

product_dir_map = {
    product_list[idx]: idx for idx in range(len(product_list))
}

product_map = {
    #unicodedata.normalize("NFC", product_list[idx][6:]): idx for idx in range(len(product_list))
    product_list[idx][product_list[idx].find('_')+1:]: idx for idx in range(len(product_list))
}

In [ ]:
import xml.etree.ElementTree as ET

def get_annot_root(root):
    # <annotation>이 루트 바로 아래, 또는 최상위 루트가 <annotation>인 경우 처리
    ann = root.find("annotation")
    return ann if ann is not None else root

def xml_to_yolo(xml_path, label_output_path):
    tree = ET.parse(xml_path)
    root = get_annot_root(tree.getroot())

    size = root.find("size")
    img_width = int(size.find("width").text)
    img_height = int(size.find("height").text)

    # 파일 내 모든 object 순회
    objects = root.findall("object")

    with open(label_output_path, "w", encoding="utf-8") as f:
        for obj in objects:
            name = obj.find("name").text

            bbox = obj.find("bndbox")
            xmin = int(bbox.find("xmin").text)
            xmax = int(bbox.find("xmax").text)
            ymin = int(bbox.find("ymin").text)
            ymax = int(bbox.find("ymax").text)

            # YOLO 형식으로 변환
            x_center = ((xmin + xmax) / 2.0) / img_width
            y_center = ((ymin + ymax) / 2.0) / img_height
            width = (xmax - xmin) / img_width
            height = (ymax - ymin) / img_height

            cls_id = product_map[name]
            f.write(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

def xml_to_pascal_voc(xml_path):
    tree = ET.parse(xml_path)
    root = get_annot_root(tree.getroot())

    labels = []
    bboxes = []
    for obj in root.findall("object"):
        name = obj.find("name").text
        bbox = obj.find("bndbox")
        xmin = int(bbox.find("xmin").text)
        xmax = int(bbox.find("xmax").text)
        ymin = int(bbox.find("ymin").text)
        ymax = int(bbox.find("ymax").text)
        labels.append(name)
        bboxes.append([xmin, ymin, xmax, ymax])

    return labels, bboxes  # [name, ...], [[xmin, ymin, xmax, ymax], ...]

def pascal_voc_to_yolo(size, boxes): # box : [[x1,y1,x2,y2], ...]
    dh = 1./size[0]
    dw = 1./size[1]

    ret = []
    for box in boxes:
        x = (box[0] + box[2])/2.0
        y = (box[1] + box[3])/2.0
        w = box[2] - box[0]
        h = box[3] - box[1]
        x = x*dw
        w = w*dw
        y = y*dh
        h = h*dh
        ret.append([x,y,w,h])
    return ret

def to_pixel_pascal_voc(boxes, W, H):
    fixed = []
    for x1, y1, x2, y2 in boxes:
        # 0~1 범위면 YOLO/정규화 좌표로 간주 → 픽셀 VOC로 환산
        if 0.0 <= x1 <= 1.0 and 0.0 <= y1 <= 1.0 and 0.0 <= x2 <= 1.0 and 0.0 <= y2 <= 1.0:
            x1, x2 = x1 * W, x2 * W
            y1, y2 = y1 * H, y2 * H

        # 순서 뒤집힌 케이스 보정
        if x2 < x1: x1, x2 = x2, x1
        if y2 < y1: y1, y2 = y2, y1

        # 경계 클리핑
        x1 = max(0, min(x1, W - 1))
        x2 = max(0, min(x2, W - 1))
        y1 = max(0, min(y1, H - 1))
        y2 = max(0, min(y2, H - 1))

        # 0 면적 방지(1px 확장)
        if x2 <= x1: x2 = min(W - 1, x1 + 1)
        if y2 <= y1: y2 = min(H - 1, y1 + 1)

        fixed.append([int(round(x1)), int(round(y1)), int(round(x2)), int(round(y2))])
    return fixed

In [ ]:
import numpy as np
import albumentations as A
import random

augmentor_light = A.Compose([
    A.MotionBlur(blur_limit=(9, 25), p=0.2),
])

augmentor_heavy = A.Compose([
    A.Affine(scale=(0.5,1.5), translate_percent=(0.0, 0.3), rotate=(0, 360), shear=0, p=0.5),
    A.CoarseDropout(num_holes_range=(1,3), hole_height_range=(0.1,0.3), hole_width_range=(0.1,0.3), p=0.2),
    A.RandomBrightnessContrast(brightness_limit=(-0.3, 0.4), contrast_limit=0, p=0.2),
    A.RandomShadow(num_shadows_limit=(1,2), shadow_dimension=5, shadow_intensity_range=(0.1,0.3), p=0.2),
    A.MotionBlur(blur_limit=(9, 25), p=0.2),
    A.ISONoise(color_shift=(0.005, 0.02), intensity=(0.05, 0.15), p=1.0),
], p=1)

def add_vignette(img, strength=0.45, power=2.2, p=0.2):
    """가장자리 어둡게 (비네팅)"""

    if random.random() > p:
        return img

    h, w = img.shape[:2]
    y, x = np.ogrid[:h, :w]
    cy, cx = h / 2, w / 2
    dist = np.sqrt((x - cx)**2 + (y - cy)**2)
    max_dist = np.sqrt(cx**2 + cy**2)
    mask = 1 - strength * (dist / max_dist) ** power
    mask = np.clip(mask, 0, 1).astype(np.float32)
    return (img.astype(np.float32) * mask[..., None]).clip(0, 255).astype(np.uint8)

In [ ]:
import numpy as np
import albumentations as A
import cv2

def augment(img_path, xml_path, augmentor):
  # 바운딩 박스 형식 지정 (Pascal VOC 형식)
  # img = cv2.imread(img_path)
  img_array = np.fromfile(img_path, np.uint8)
  img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
  labels, bboxes = xml_to_pascal_voc(xml_path)

  # 정규화/YOLO 값이 섞여 있어서 픽셀 VOC로 강제 통일
  bboxes = to_pixel_pascal_voc(bboxes, img.shape[1], img.shape[0])

  transform = A.Compose([
    A.LongestMaxSize(max_size=640),
    augmentor,
    ],
    bbox_params=A.BboxParams(
      format='pascal_voc',
      label_fields=['category_ids'],
      min_visibility=0.01,
      clip=True,
    )
  )

  # 증강 적용
  augmented = transform(image=img, bboxes=bboxes, category_ids=labels)
  img_aug = add_vignette(augmented['image'], p=0)

  if len(augmented['bboxes']) == 0:
    return labels, img, []  # 바운딩 박스 예외 처리
  return labels, img_aug, augmented['bboxes']

In [7]:
### augmentor 테스트
import cv2
import numpy as np

img_array = np.fromfile('10102_30_s_2.jpg', np.uint8)
img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

_, img, _ = augment('10102_30_s_2.jpg', '10102_30_s_2_meta.xml', augmentor_light)

cv2.imshow('window', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
import cv2
import shutil

def process_dataset(pairs, split):
  for img_path, xml_path in pairs:
    filename = os.path.basename(img_path)
    dst_img = f"{dir_root}/processed/images/{split}/"
    dst_label = f"{dir_root}/processed/labels/{split}/"

    if(split == 'train'):
      # 증강 없이 복사
      # shutil.copy(img_path, dst_img + filename[:-4]+"_0"+filename[-4:])
      # xml_to_yolo(xml_path, dst_label + filename[:-4]+"_0.txt")

      labels, augimg, augboxes = augment(img_path, xml_path, augmentor_light)
      if(len(augboxes) == 0):
        continue

      cv2.imwrite(dst_img + filename[:-4]+"_1"+filename[-4:], augimg)
      bboxes = pascal_voc_to_yolo(augimg.shape[:2], augboxes)
      with open(dst_label + filename[:-4]+"_1.txt", "w") as f:
        for label, bbox in zip(labels, bboxes):
          f.write(f"{product_map[label]} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")
    elif(split == 'val'):
      xml_to_yolo(xml_path, dst_label+ f"{filename.replace('.jpg', '.txt')}")
      shutil.copy(img_path, dst_img + f"{filename}")

In [9]:
from glob import glob

def set_dataset(image_root, label_root):
  image_label_pairs = []
  for dir_name in product_dir_map.keys():
    image_dir = os.path.join(image_root, dir_name)
    label_dir = os.path.join(label_root, dir_name)

    image_files = glob(os.path.join(image_dir, "*.jpg"))

    for img_path in image_files:
      filename = os.path.basename(img_path)
      xml_path = os.path.join(label_dir, filename.replace(".jpg", "_meta.xml"))
      image_label_pairs.append((img_path, xml_path))
  return image_label_pairs

In [11]:
train_pairs = set_dataset(image_root, label_root)
val_pairs = set_dataset(val_image_root, val_label_root)

process_dataset(train_pairs, 'train')
process_dataset(val_pairs, 'val')

In [ ]:
### 비분류 xml 파일을 서브 폴더로 정리
import os
import shutil

# 경로 설정
image_root = r"C:/capstone/data/extra/image"
label_root = r"C:/capstone/data/extra/label"

# image_root 내 모든 서브폴더 탐색
for subfolder in os.listdir(image_root):
    subfolder_path = os.path.join(image_root, subfolder)
    if not os.path.isdir(subfolder_path):
        continue  # 폴더가 아니면 건너뜀

    # label에 동일 이름의 서브폴더 생성
    target_label_subfolder = os.path.join(label_root, subfolder)
    os.makedirs(target_label_subfolder, exist_ok=True)

    # 서브폴더 내 jpg 파일 확인
    for file in os.listdir(subfolder_path):
        if file.lower().endswith(".jpg"):
            base_name = os.path.splitext(file)[0]
            xml_file = base_name + ".xml"
            xml_path = os.path.join(label_root, xml_file)

            if os.path.exists(xml_path):
                # xml 파일을 대상 서브폴더로 이동
                shutil.move(xml_path, os.path.join(target_label_subfolder, xml_file))
                print(f"Moved: {xml_file} -> {target_label_subfolder}")
            else:
                print(f"Warning: {xml_file} not found in {label_root}")

In [ ]:
### 라벨 데이터 파일명에 _meta 추가
import os

# 대상 루트 경로
root_dir = r"C:\capstone\data\extra\labels"

# 모든 하위 폴더 순회
for folder_path, _, files in os.walk(root_dir):
    for file_name in files:
        # 확장자 분리
        name, ext = os.path.splitext(file_name)
        new_name = f"{name}_meta{ext}"
        
        old_path = os.path.join(folder_path, file_name)
        new_path = os.path.join(folder_path, new_name)

        # 이미 '_meta'가 붙은 파일은 건너뛰기
        if "_meta" in name:
            continue
        
        # 이름 변경
        os.rename(old_path, new_path)
        print(f"Renamed: {old_path} → {new_path}")